<a href="https://colab.research.google.com/github/mansamoussa/zebrAssist/blob/master/DemoYolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Swiss Zebra Crossing & Traffic Light Detection - YOLOv8 Training
# Run this in Google Colab with GPU enabled

# =============================================================================
# 1. SETUP AND INSTALLATIONS
# =============================================================================

# Check if we're in Colab and enable GPU
import os
try:
    from google.colab import drive
    IN_COLAB = True
    print("✅ Running in Google Colab")

    # Mount Google Drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted")

    # Check GPU
    !nvidia-smi

except ImportError:
    IN_COLAB = False
    print("⚠️ Not in Colab - make sure you have GPU available")

# Install YOLOv8
!pip install ultralytics


✅ Running in Google Colab
Mounted at /content/drive
✅ Google Drive mounted
/bin/bash: line 1: nvidia-smi: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00


In [ ]:

# Import libraries
import yaml
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
from IPython.display import Image, display
import shutil

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ PyTorch version: 2.8.0+cu126
✅ CUDA available: False


In [ ]:

# =============================================================================
# 2. DOWNLOAD AND EXTRACT DATASET
# =============================================================================

# Find the dataset directory (adjust if needed)
dataset_path = "/content/drive/MyDrive/Computer_Vision_Project/data/zebrAssit.v1i.yolov8"
data_yaml = f"{dataset_path}/data.yaml"

# Verify dataset
with open(data_yaml, 'r') as f:
    data_config = yaml.safe_load(f)

print(f"Dataset classes: {data_config['names']}")
print(f"Number of classes: {data_config['nc']}")

# Fix paths in data.yaml for Colab
data_config['train'] = f"{dataset_path}/train/images"
data_config['val'] = f"{dataset_path}/valid/images" if os.path.exists(f"{dataset_path}/valid") else f"{dataset_path}/train/images"
data_config['test'] = f"{dataset_path}/test/images" if os.path.exists(f"{dataset_path}/test") else f"{dataset_path}/train/images"

# Save updated config
with open(data_yaml, 'w') as f:
    yaml.dump(data_config, f)

print("✅ Dataset paths updated for Colab")


Dataset classes: ['traffic_light_green', 'traffic_light_red', 'traffic_light_yellow', 'zebra_crossing']
Number of classes: 4
✅ Dataset paths updated for Colab


In [ ]:

# =============================================================================
# 3. TRAIN THE MODEL
# =============================================================================

# Initialize YOLOv8 model
model = YOLO('yolov8n.pt')  # nano version for speed

print("🚀 Starting training...")

# Train the model
results = model.train(
    data=data_yaml,
    epochs=10,          # Increase for better results
    imgsz=640,
    batch=16,            # Adjust based on GPU memory
    name='swiss_zebra_traffic',
    patience=5,         # Early stopping
    save=True,
    device=0 if torch.cuda.is_available() else 'cpu'
)

print("✅ Training completed!")


🚀 Starting training...
Ultralytics 8.3.186 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Computer_Vision_Project/data/zebrAssit.v1i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=swiss_zebra_traffic2, nbs=64, nms=False, opset=None, optimize=False, optimizer=a

In [ ]:

# =============================================================================
# 4. EVALUATE THE MODEL
# =============================================================================

# Load the best model
best_model = YOLO('/content/runs/detect/swiss_zebra_traffic/weights/best.pt')

# Validate the model
validation_results = best_model.val()

print(f"mAP50: {validation_results.box.map50:.3f}")
print(f"mAP50-95: {validation_results.box.map:.3f}")


Ultralytics 8.3.186 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 1.1±0.6 ms, read: 37.0±6.7 MB/s, size: 78.0 KB)
val: Scanning /content/drive/MyDrive/Computer_Vision_Project/data/zebrAssit.v1i.yolov8/train/labels.cache... 22 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 22/22 6829.1it/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 12, len(boxes) = 44. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.37it/s 5.5s
                   all         22         44    0.00321       0.29     0.0047    0.00146
   traffic_light_green          8          9    0.00123      0.222    0.00319    0.00181


In [2]:

# =============================================================================
# 5. TEST ON SAMPLE IMAGES
# =============================================================================

# Test on a few sample images
test_images = []
train_img_dir = f"{dataset_path}/train/images"

# Get first 3 images for testing
for img_file in os.listdir(train_img_dir)[:3]:
    if img_file.endswith(('.jpg', '.jpeg', '.png')):
        test_images.append(os.path.join(train_img_dir, img_file))

print(f"Testing on {len(test_images)} sample images...")

# Run inference
for img_path in test_images:
    results = best_model(img_path)

    # Plot results
    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image

        print(f"Results for {os.path.basename(img_path)}:")
        display(im)

        # Print detected objects
        if r.boxes is not None:
            for box in r.boxes:
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = data_config['names'][class_id]
                print(f"  - {class_name}: {confidence:.2f}")
        print("-" * 50)



NameError: name 'dataset_path' is not defined

In [ ]:

# =============================================================================
# 6. EXPORT MODEL FOR DEPLOYMENT
# =============================================================================

# Export to different formats
print("📦 Exporting model for deployment...")

# ONNX format (good for mobile deployment)
best_model.export(format='onnx')
print("✅ ONNX model exported")

# TensorRT (if you have compatible GPU)
try:
    best_model.export(format='engine')
    print("✅ TensorRT model exported")
except:
    print("⚠️ TensorRT export failed (normal if no compatible GPU)")


In [ ]:

# =============================================================================
# 7. DOWNLOAD RESULTS
# =============================================================================

# Zip the results for download
!zip -r swiss_zebra_results.zip /content/runs/detect/swiss_zebra_traffic/

print("✅ Results zipped!")
print("📁 Download 'swiss_zebra_results.zip' from Colab files panel")
print("🎯 Your trained model is: runs/detect/swiss_zebra_traffic/weights/best.pt")

# Display training plots
training_plots = [
    '/content/runs/detect/swiss_zebra_traffic/results.png',
    '/content/runs/detect/swiss_zebra_traffic/confusion_matrix.png'
]

for plot_path in training_plots:
    if os.path.exists(plot_path):
        print(f"\n{os.path.basename(plot_path)}:")
        display(Image(plot_path))


In [ ]:

# =============================================================================
# 8. SIMPLE INFERENCE FUNCTION
# =============================================================================

def detect_zebra_and_lights(image_path, confidence_threshold=0.5):
    """Simple function to detect zebra crossings and traffic lights."""

    results = best_model(image_path, conf=confidence_threshold)

    detections = []
    for r in results:
        if r.boxes is not None:
            for box in r.boxes:
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = data_config['names'][class_id]

                # Get bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0].tolist()

                detections.append({
                    'class': class_name,
                    'confidence': confidence,
                    'bbox': [x1, y1, x2, y2]
                })

    return detections

print("✅ Inference function ready!")
print("Use detect_zebra_and_lights('path_to_image.jpg') to test new images")